In [1]:
# -*- coding: utf-8 -*-

# Recurrent Neural Network

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Importing the training set
df = pd.read_csv('timeStep8CorpusFull.csv',encoding='utf-8')

df = df.head(300000)

#X and y
X = df[['0','1','2','3','4','5','6','7']]
y = df[['isEnd']]

#Label each character
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X[['0']])
X = X.apply(le.transform)



trainPer= 0.8
boundary = int(X.shape[0]*trainPer)
X_train = X.iloc[:boundary].values
X_test = X.iloc[boundary:].values
y_train = y.iloc[:boundary].values
y_test = y.iloc[boundary:].values



timeStep=8

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]))

maxChar = 65
X_train = (np.arange(maxChar) == X_train[:,:,None]-1).astype(int)
X_test = (np.arange(maxChar) == X_test[:,:,None]-1).astype(int)


#--------------------------- RNN--------------------------------------#

# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense,Bidirectional
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
classifier = Sequential()

# Adding the input layer and the LSTM layer
classifier.add(LSTM(units = 10, return_sequences = True, input_shape = (None, X_train.shape[2])))
classifier.add(Dropout(0.2))
# Adding a second LSTM layer
classifier.add(Bidirectional(LSTM(units = 10, return_sequences = True)))
classifier.add(Dropout(0.2))

# Adding a second LSTM layer
classifier.add(Bidirectional(LSTM(units = 10)))
classifier.add(Dropout(0.2))

## Adding a fourth LSTM layer
#classifier.add(LSTM(units = 10))

# Adding the output layer
classifier.add(Dense(units = 1))

# Compiling the RNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

# Fitting the RNN to the Training set
classifier.fit(X_train, y_train, batch_size = 32, epochs = 200)
# Part 3 - Making the predictions and visualising the results


classifier.save('thaiseg_300krows_8timestep_bidirection.h5')
y_pred = classifier.predict(X_test)
y_pred = y_pred>0.5
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average='macro'))




def thaiSegment(word):
    test = word[-timeStep+1:]+word #Same trick as in CNN
    df = {}
    for i in range(timeStep):
        if(i-timeStep+1<0):
            df[i] = list(test[i:i-timeStep+1])
        else:
            df[i] = list(test[i:])
    df_test = pd.DataFrame(df)
    df_test = df_test.apply(le.transform).values
    df_test = (np.arange(maxChar) == df_test[:,:,None]-1).astype(int)
    y_pred = classifier.predict(df_test)
    y_pred = y_pred>0.5
    
    df_test  = pd.DataFrame(df)
    df_test['isEnd'] = y_pred
    
    words = []
    w = ''
    
    for i in range(len(test)-timeStep+1):
        c = test[i+timeStep-1]
        w+= c
        if(y_pred[i][0]==True):
            words.append(w)
            w=''
            
    words.append(w)
    if "" in words: 
        words.remove('')
    return "|".join(words)


print(thaiSegment('สมชายจะไปหาหมอที่ตลาด'))
print(thaiSegment('ไก่จิกเด็กตายบนปากโอง'))
print(thaiSegment('การประชุมที่ธรรมศาสตร์รังสิต'))
print(thaiSegment('การออกรบต้องรู้ว่าจะทำอะไร'))

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Using TensorFlow backend.


InternalError: Failed to create session.